In [2]:
import pandas as pd 
import numpy as np
import os 


In [3]:
pd.set_option("display.max_columns",85)
# for displaying purposes 

## Reading the Data
<hr>

In [18]:
df1= pd.read_csv("season2017-2018.csv")# first resource
df2= pd.read_csv("season2017-2018(2).csv") # second resource

# the data is scraped from different resourses
# so the first part of this notebook will be for cleaing the data and 
# unify the attributes and so on

In [19]:
df1.drop(["Unnamed: 0"],axis='columns',inplace=True)
# droping the indexing column in the data 

In [20]:
# convert the date column from string to a date object 
# date column will be a very important column in the preprocssesing process 
# it will be the index that the matches will be jonined at 
# the date should be the same for the same match in different resources 
# it means that the data will be sorted on the date then on other attributes discussed later 
df1["date"]=pd.to_datetime(df1["date"].str.replace("-",""),format="%Y%m%d")

In [21]:
df2["Date"]=pd.to_datetime(df2["Date"].str.replace("/",""),format="%d%m%Y")

In [22]:
# dropping unwanted columns 

In [23]:
df2.drop(["Div","FTHG","FTAG","HTHG","HTAG","HTR","Referee"],axis='columns',inplace=True)
# drop the dive , FTHG(final time home goal), FTAG(Final time  away Goals)
# HTGH(half time goals home ), " ... " "to referee"

In [24]:
index = df2.columns.get_loc("B365H")
df2.drop(df2.iloc[:,index:],axis=1,inplace=True)
# dropping the betting data 

In [25]:
df2.sort_values(by=["Date","HomeTeam","AwayTeam"],inplace=True)
# sorting the data on Date , Home , then Away 
# the data relevant to each other 
# it means that the 380 row in the first resources will be the same 
# matches for the second resource which will make the joining process correct 

In [26]:
df1.sort_values(by=["date","home","away"],inplace=True)

In [27]:
# TCP -> Team Complete Pass
# TTP -> Team Total Pass

df1["TCP"]=df1["passeshome"].str.split("—").apply(lambda x : x[0].split("of")[0])
df1["TTP"]=df1["passeshome"].str.split("—").apply(lambda x : x[0].split("of")[1])
df1.drop(["passeshome"],axis=1,inplace=True)


# the date was in string format like that "200 of 400-- 50%"
# so I want to get the team complete pass 200 , team total pass 400 


In [28]:
#OCP ->Opponent Complete Pass
#OTP -> Opponent Total Pass
df1["OCP"]=df1["passesaway"].str.split("—").apply(lambda x : x[1].split("of")[0])
df1["OTP"]=df1["passesaway"].str.split("—").apply(lambda x : x[1].split("of")[1])
df1.drop(["passesaway"],axis=1,inplace=True)

# the same thing in opponent 

In [29]:
df1.drop(["saveshome","savesaway"],axis=1,inplace=True)

# you have extracted the data so , drop the columns 

In [30]:
df1.rename(columns={"possessionhome":"POT","possessionaway":"POO","crosseshome":"CRT",
                   "crossesaway":"CRO","toucheshome":"TCT","touchesaway":"TCO",
                   "tackleshome":"TTT","tacklesaway":"TTO","interceptionhome":"IPT","interceptionaway":"IPO",
                   "aerialshome":"ART","aerialsaway":"ARO","clearnacehome":"CLT","clearnaceaway":"CLO"},inplace=True)


# renaming the date 

# POT-> PossesionTeam
# POO=> PossesionOpponent
# CRT => crossesTeam
# CRO => crossesOpponent
# TCT => touchesTeam
# TCO => touchesOpponent
# TTT => tacklesTeam
# TTO => tacklesOpponent
# IPT => interceptionTeam
# IPO => interceptionOpponent
# ART => aerialsTeam
# ARO => aerialsOpponent
# CLT => clearnaceTeam
# CLO => clearnaceOpponent


# the team data means the home team , opponent will be the away team 
# I will explain in details what will I do to make use of the Home attribute 
# in the model 

In [31]:
df1.set_index("date",inplace=True)
df2.set_index("Date",inplace=True)

In [32]:
total_data=pd.concat([df1,df2],axis=1).drop(["HomeTeam","AwayTeam"],axis=1)

# merging the data 

In [33]:
# TS=> Team shot 
# OS=> Oppoent shot 
# TST=>Team shot taret
# OST=>Opponent shot target

# and the rest just change the home t- team , and away to opponent 

total_data.rename({"HS":"TS","AS":"OS","HST":"TST","AST":"OST","HF":"TF","AF":"OF","HC":"TC","AC":"OC","HY":"TY"
                   ,"AY":"OY","HR":"TR","AR":"OR"},axis=1,inplace=True)

In [34]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import copy 
# this process should be done after concat all the data exampels 
# because as you know there are 5 teams that will be present from the 
# Div1 and five will go from Div0 to Div1 each season 
# so to get all the name of the teams that 
# This is not always ideal for ML as the integers have different numerical values, 
#suggesting that one is bigger than the other, with, for example Pear > Apple, 
#which is not at all the case. To not introduce this kind of problem you'd want to use OneHotEncoder.


In [35]:
total_data.rename({"home":"team","away":"opponent"},axis=1,inplace=True)
# renameing the home to team and the away to opponent 

In [36]:
total_data["Home"]=1



## Important Part 
<hr>
**here I will  give the model the name of the team then the opponent name and the most effective attribute is 'are you palying on your home or not' so I should duplicate the data and reverse the order**
example 


- if the home team is arsenal and the away is Liverpool
- the first example should be like that {"team":"Arsenal","opponent":"Liverpool",...."Home":"1"}
-  the last input was taking in Arsenal perspective so what about liverpool
-  we should add another example in the data which will talk about Liverpool perspective 
-  {"team":"Liverpool", "opponent":"Aresnal",.... "Home":"0"}

In [37]:
temp=copy.deepcopy(total_data)

In [38]:
temp.rename({ "team":"opponent","opponent":"team" , "POT":"POO","POO":"POT" ,"CRT":"CRO","CRO":"CRT",
              "TCT":"TCO","TCO":"TCT" , "TTT":"TTO","TTO":"TTT" , "IPT":"IPO","IPO":"IPT" 
             ,"ART":"ARO","ARO":"ART", "CLT":"CLO","CLO":"CLT", "TCP":"OCP","OCP":"TCP" 
             ,"TTP":"OTP","OTP":"TTP",  "TS":"OS","OS":"TS", "TST":"OST","OST":"TST" ,"TF":"OF","OF":"TF",
            "TC":"OC","OC":"TC", "TY":"OY","OY":"TY", "TR":"OR","OR":"TR" }
            ,axis=1,inplace=True)



# just reversing the order of all the attributes the team will be the 
# opponent and the opponent will be the team to see the prespective 
# of the two clubs not only one 

In [39]:
temp=temp[["team","opponent","POT","POO","CRT","CRO","TCT","TCO","TTT","TTO","IPT","IPO","ART","ARO","CLT","CLO","TCP","TTP","OCP","OTP","FTR","TS","OS","TST","OST","TF","OF","TC","OC","TY","OY","TR","OR","Home"]]

# reording the columns to be in the  same order as the total data  

In [40]:
total_data.columns==temp.columns

filt_win= (total_data["FTR"]=='H')
filt_draw=(total_data["FTR"]=='D')
filt_loss=(total_data["FTR"]=='A')
print(total_data.loc[filt_draw].shape[0])
print(total_data.loc[filt_win].shape[0])
print(total_data.loc[filt_loss].shape[0])
total_data.loc[filt_win,"FTR"]=2
total_data.loc[filt_draw,"FTR"]=1
total_data.loc[filt_loss,"FTR"]=0
total_data["Home"]=1
total_data[["team","opponent","FTR","Home"]]

# just encode the H-> my perspective team win to 2 
#                 D-> Drawing 
#                 L-> loss 
# and see some data about the effect

99
173
108


,team,opponent,FTR,Home
2017-08-11,Arsenal,Leicester City,2,1
2017-08-12,Brighton & Hove Albion,Manchester City,0,1
2017-08-12,Chelsea,Burnley,0,1
2017-08-12,Crystal Palace,Huddersfield Town,0,1
2017-08-12,Everton,Stoke City,2,1
...,...,...,...,...
2018-05-13,Newcastle United,Chelsea,2,1
2018-05-13,Southampton,Manchester City,0,1
2018-05-13,Swansea City,Stoke City,0,1
2018-05-13,Tottenham Hotspur,Leicester City,2,1
